## Cargando datos en memoria

La carga de API es fundamental para muchas operaciones nilmtk y proporciona una gran flexibilidad. Veamos las formas en que podemos cargar datos de un NILMTK DataStore en la memoria. Para ver la gama completa de consultas posibles, usaremos el conjunto de datos [iAWE data set](http://iawe.github.io) (cuyo archivo HDF5 se puede descargar [aqui](https://copy.com/C2sIt1UfDx1mfPlC)).

La función de carga devuelve un generador de DataFrames cargado desde el DataStore en función de las condiciones especificadas. Si no se especifican condiciones, se cargan todos los datos de todas las columnas. (Si no ha encontrado generadores de Python, puede que valga la pena leer esta guía rápida de generadores de Python [this quick guide to Python generators](http://stackoverflow.com/a/1756156/732596).)).

**NOTA**: Si está en Windows, recuerde evitar las barras diagonales inversas, usar barras inclinadas hacia adelante o usar cadenas sin formato al pasar rutas en Python, p. Ej. uno de los siguientes funcionaría:


```python
iawe = DataSet('c:\\data\\iawe.h5')
iawe = DataSet('c:/data/iawe.h5')
iawe = DataSet(r'c:\data\iawe.h5')
```

In [1]:
from nilmtk import DataSet

iawe = DataSet('dsual.h5')
elec = iawe.buildings[1].elec
elec

MeterGroup(meters=
  ElecMeter(instance=1, building=1, dataset='DSUAL', site_meter, appliances=[])
  ElecMeter(instance=2, building=1, dataset='DSUAL', appliances=[Appliance(type='boiler', instance=1)])
  ElecMeter(instance=3, building=1, dataset='DSUAL', appliances=[Appliance(type='fan', instance=1)])
  ElecMeter(instance=4, building=1, dataset='DSUAL', appliances=[Appliance(type='freezer', instance=1)])
  ElecMeter(instance=5, building=1, dataset='DSUAL', appliances=[Appliance(type='television', instance=1)])
  ElecMeter(instance=6, building=1, dataset='DSUAL', appliances=[Appliance(type='vacuum cleaner', instance=1)])
  ElecMeter(instance=7, building=1, dataset='DSUAL', appliances=[Appliance(type='light', instance=1)])
)

Veamos qué medidas tenemos para el frigorífico:

In [2]:
fridge = elec['fridge']
fridge.available_columns()

[('frequency', None),
 ('power', 'active'),
 ('current', None),
 ('power factor', None),
 ('power', 'apparent'),
 ('power', 'reactive'),
 ('voltage', None)]

## Cargando datos

### Carga de todas las columnas (por defecto)

In [3]:
df = next(fridge.load())
df.head()

physical_quantity          frequency     power   current     power            \
type                                    active            apparent  reactive   
2021-11-11 17:49:47+01:00  50.011902  3.020910  0.023590  3.727449  2.183570   
2021-11-11 17:49:48+01:00  50.011162  2.813174  0.029873  4.126142  3.014136   
2021-11-11 17:49:49+01:00  50.010120  2.804554  0.026298  3.962116  2.796514   
2021-11-11 17:49:50+01:00  50.009682  2.941298  0.026703  4.000784  2.705020   
2021-11-11 17:49:51+01:00  50.009441  2.854542  0.026253  3.842267  2.557182   

physical_quantity             voltage  
type                                   
2021-11-11 17:49:47+01:00  238.014008  
2021-11-11 17:49:48+01:00  238.138992  
2021-11-11 17:49:49+01:00  238.162201  
2021-11-11 17:49:50+01:00  238.033401  
2021-11-11 17:49:51+01:00  238.013199

### Carga una columna simple de datos de  potencia

Usar `fridge.power_series()` que devuelve un generador de objetos pandas.Series unidimensionales, cada uno de los cuales contiene datos de energía utilizando el tipo de CA más 'sensible':

In [4]:
series = next(fridge.power_series())
series.head()

2021-11-11 17:49:47+01:00    3.020910
2021-11-11 17:49:48+01:00    2.813174
2021-11-11 17:49:49+01:00    2.804554
2021-11-11 17:49:50+01:00    2.941298
2021-11-11 17:49:51+01:00    2.854542
Name: (power, active), dtype: float32

o, para obtener potencia reactiva:

In [5]:
series = next(fridge.power_series(ac_type='reactive'))
series.head()

2021-11-11 17:49:47+01:00    2.183570
2021-11-11 17:49:48+01:00    3.014136
2021-11-11 17:49:49+01:00    2.796514
2021-11-11 17:49:50+01:00    2.705020
2021-11-11 17:49:51+01:00    2.557182
Name: (power, reactive), dtype: float32

### Especificar cantidad_física o tipo de CA

In [6]:
df = next(fridge.load(physical_quantity='power', ac_type='reactive'))
df.head()

physical_quantity,power
type,reactive
2021-11-11 17:49:47+01:00,2.183570
2021-11-11 17:49:48+01:00,3.014136
2021-11-11 17:49:49+01:00,2.796514
2021-11-11 17:49:50+01:00,2.705020
2021-11-11 17:49:51+01:00,2.557182


Para cargar datos de voltaje:

In [7]:
df = next(fridge.load(physical_quantity='voltage'))
df.head()

physical_quantity,voltage
type,
2021-11-11 17:49:47+01:00,238.014008
2021-11-11 17:49:48+01:00,238.138992
2021-11-11 17:49:49+01:00,238.162201
2021-11-11 17:49:50+01:00,238.033401
2021-11-11 17:49:51+01:00,238.013199


In [8]:
df = next(fridge.load(physical_quantity = 'power'))
df.head()

physical_quantity             power                    
type                         active  apparent  reactive
2021-11-11 17:49:47+01:00  3.020910  3.727449  2.183570
2021-11-11 17:49:48+01:00  2.813174  4.126142  3.014136
2021-11-11 17:49:49+01:00  2.804554  3.962116  2.796514
2021-11-11 17:49:50+01:00  2.941298  4.000784  2.705020
2021-11-11 17:49:51+01:00  2.854542  3.842267  2.557182

### Cargando especificando el tipo de CA

In [9]:
df = next(fridge.load(ac_type='active'))
df.head()

physical_quantity,power
type,active
2021-11-11 17:49:47+01:00,3.020910
2021-11-11 17:49:48+01:00,2.813174
2021-11-11 17:49:49+01:00,2.804554
2021-11-11 17:49:50+01:00,2.941298
2021-11-11 17:49:51+01:00,2.854542


### Cargando por remuestreo a un período específico

In [10]:
# resample por minutos (i.e. con un simple periodo de  60 secs)
df = next(fridge.load(ac_type='active', sample_period=60))
df.head()

physical_quantity,power
type,active
2021-11-11 17:49:00+01:00,26.569918
2021-11-11 17:50:00+01:00,49.535671
2021-11-11 17:51:00+01:00,45.595875
2021-11-11 17:52:00+01:00,44.999367
2021-11-11 17:53:00+01:00,44.706043
